# Eurostat bioenergy balance (June 2021 edition)

Extract bioenergy related data from an archive containing XLSB files, one for each EU country which contain multiple sheets for each year (1990-2019).

Data from Eurostat's [energy balances](https://ec.europa.eu/eurostat/web/energy/data/energy-balances) June 2021 edition.

Walk through excel files (country spreadsheets) and parse selected variables and fuels for each year (sheet in country's spreadsheet).

Somewhere on Eurostat's website there might be a better source for parsing this data, but I did not find it.

In [1]:
import os
import zipfile
import requests
import pandas as pd
import numpy as np
import pyxlsb

In [2]:
def parse_values_for_country(file, country, variables, fuels):
    """Reads fuel variable in multiple sheets 2002-2018.
    Sums the values across multiple columns if relevant.
    Returns: dict
    """
    country_data = {}
    
    for year in range(2002,2020):
        df = pd.read_excel(
            file,
            engine='pyxlsb',
            sheet_name=str(year),
            skiprows=[0,1,2,3],
            index_col=1,
            na_values=':',
            )
        for variable in variables:
            for fuel, start, end in fuels:             
                try:
                    country_data[(country, year, fuel, variable.lower().replace(' ', '_'))] = df.loc[variable, start:end].sum()
                except TypeError:
                    country_data[(country, year, fuel, variable.lower().replace(' ', '_'))] = pd.to_numeric(df.loc[variable, start:end], errors='coerce').sum()

    return country_data

In [3]:
def walk_through_excel_files(directory, variables, fuels):
    d = {}
    
    for filename in os.listdir(directory):
        if '!' not in filename and '.pdf' not in filename: # skip readme files 
            country = filename.split('-')[0]
            excel_path = os.path.join(directory, filename)
            data = parse_values_for_country(excel_path, country, variables, fuels)
            d.update(data)
    return d

In [4]:
# Selected variables for bioenergy and some other for context
variables = [
'Primary production',
'Imports',
'Exports',
'Gross inland consumption',
]

fuels = [
    ('total', 'Total', 'Total'),
    ('renewables', 'Renewables and biofuels', 'Renewables and biofuels'),
    ('bioenergy', 'Bioenergy', 'Bioenergy',),
    ('solid_biomass', 'Primary solid biofuels', 'Primary solid biofuels'),
    ('biofuels', 'Pure biogasoline', 'Other liquid biofuels'),
    ('biogas', 'Biogases', 'Biogases'),
    ('ren_mun_waste', 'Renewable municipal waste', 'Renewable municipal waste'),
    ]

In [5]:
# The url contains February, but it is the June file
url = 'https://ec.europa.eu/eurostat/documents/38154/4956218/Energy-balance-sheets-February-2021-edition.zip/4b1d6665-f303-be7d-a7e5-1e0da16ec0d9?t=1612709565471'

r = requests.get(url)

with open('eurostat_balances_2021.zip', 'wb') as f:
    f.write(r.content)

In [6]:
with zipfile.ZipFile('eurostat_balances_2021.zip', 'r') as zip_archive:
    zip_archive.extractall(path='balances/')

In [7]:
# This is quite slow, opening many files, one time for each sheet
# There must be a better way

%time data_dict = walk_through_excel_files('balances/', variables, fuels)

CPU times: user 2min 7s, sys: 2.54 s, total: 2min 9s
Wall time: 2min 23s


In [8]:
# https://stackoverflow.com/questions/44012099/creating-a-dataframe-from-a-dict-where-keys-are-tuples
df1 = pd.Series(data_dict).reset_index()
df1.columns = ['country', 'year', 'fuel', 'variable', 'value']

In [9]:
df1.head(3)

,country,year,fuel,variable,value
0,AT,2002,total,primary_production,9831.767
1,AT,2002,renewables,primary_production,6490.482
2,AT,2002,bioenergy,primary_production,2920.414


In [10]:
df2 = df1.set_index(['country', 'year', 'fuel', 'variable']).unstack(level=3)

In [11]:
df2.head(3)

value                                   \
variable               exports gross_inland_consumption imports   
country year fuel                                                 
AL      2002 bioenergy     0.0                  255.995     0.0   
             biofuels      0.0                    0.000     0.0   
             biogas        0.0                    0.000     0.0   

                                           
variable               primary_production  
country year fuel                          
AL      2002 bioenergy            255.995  
             biofuels               0.000  
             biogas                 0.000

In [12]:
df2.columns = df2.columns.droplevel(0).values

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5418 entries, ('AL', 2002, 'bioenergy') to ('XK', 2019, 'total')
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   exports                   5418 non-null   float64
 1   gross_inland_consumption  5418 non-null   float64
 2   imports                   5418 non-null   float64
 3   primary_production        5418 non-null   float64
dtypes: float64(4)
memory usage: 188.1+ KB


In [14]:
df2.sort_index(ascending=True, inplace=True)

In [15]:
df2['dependency'] = (df2['imports'] - df2['exports']) / df2['gross_inland_consumption']
df2

exports  gross_inland_consumption  imports  \
country year fuel                                                        
AL      2002 bioenergy        0.000                   255.995    0.000   
             biofuels         0.000                     0.000    0.000   
             biogas           0.000                     0.000    0.000   
             ren_mun_waste    0.000                     0.000    0.000   
             renewables       0.000                   559.382    0.000   
...                             ...                       ...      ...   
XK      2019 biogas           0.000                     0.000    0.000   
             ren_mun_waste    0.000                     0.000    0.000   
             renewables       0.042                   402.360   55.816   
             solid_biomass    0.042                   374.978   55.816   
             total           94.358                  2668.905  909.275   

                            primary_production  dependency  
country year fuel                                           
AL      2002 bioenergy                 255.995    0.000000  
             biofuels                    0.000         NaN  
             biogas                      0.000         NaN  
             ren_mun_waste               0.000         NaN  
             renewables                559.382    0.000000  
...                                        ...         ...  
XK      2019 biogas                      0.000         NaN  
             ren_mun_waste               0.000         NaN  
             renewables                346.586    0.138617  
             solid_biomass             319.205    0.148739  
             total                    1849.116    0.305338  

[5418 rows x 5 columns]

In [16]:
df2.to_csv(
    'balances_bioenergy_2002_2019_ktoe.csv',
    decimal=',',
    )

In [17]:
df3 = df2.copy()

In [18]:
tj_ktoe = 41.868

df3 = df3.loc[:, 'exports': 'primary_production'] * tj_ktoe

# Keep the share based on the original data in ktoe
df3['dependency'] = df2['dependency']
df3

exports  gross_inland_consumption  \
country year fuel                                                   
AL      2002 bioenergy         0.000000              10717.998660   
             biofuels          0.000000                  0.000000   
             biogas            0.000000                  0.000000   
             ren_mun_waste     0.000000                  0.000000   
             renewables        0.000000              23420.205576   
...                                 ...                       ...   
XK      2019 biogas            0.000000                  0.000000   
             ren_mun_waste     0.000000                  0.000000   
             renewables        1.758456              16846.008480   
             solid_biomass     1.758456              15699.578904   
             total          3950.580744             111741.714540   

                                 imports  primary_production  dependency  
country year fuel                                                         
AL      2002 bioenergy          0.000000        10717.998660    0.000000  
             biofuels           0.000000            0.000000         NaN  
             biogas             0.000000            0.000000         NaN  
             ren_mun_waste      0.000000            0.000000         NaN  
             renewables         0.000000        23420.205576    0.000000  
...                                  ...                 ...         ...  
XK      2019 biogas             0.000000            0.000000         NaN  
             ren_mun_waste      0.000000            0.000000         NaN  
             renewables      2336.904288        14510.862648    0.138617  
             solid_biomass   2336.904288        13364.474940    0.148739  
             total          38069.525700        77418.788688    0.305338  

[5418 rows x 5 columns]

In [19]:
df3.to_csv(
    'balances_bioenergy_2002_2019_tj.csv',
    decimal=',',
    )

In [20]:
# Some minimal testing
idx = pd.IndexSlice

In [21]:
df2.loc[idx['CZ', 2018, 'bioenergy'], ['exports']]

exports    549.453
Name: (CZ, 2018, bioenergy), dtype: float64

In [22]:
assert df2.loc[idx['CZ', 2018, 'bioenergy'], ['exports']].item() == 549.453

In [23]:
df2.loc[idx['CZ', 2009, 'bioenergy'], ['primary_production']]

primary_production    2761.8
Name: (CZ, 2009, bioenergy), dtype: float64

In [24]:
assert df2.loc[idx['CZ', 2009, 'bioenergy'], ['primary_production']].item() == 2761.8

In [25]:
result_cz_2009_bioenergy = df2.loc[idx['CZ', 2009, 'bioenergy']]
result_cz_2009_bioenergy

exports                      318.821000
gross_inland_consumption    2568.609000
imports                      123.617000
primary_production          2761.800000
dependency                    -0.075996
Name: (CZ, 2009, bioenergy), dtype: float64

In [26]:
cz_2009_bioenergy = pd.Series(
    {'exports': 318.821,
     'gross_inland_consumption': 2568.609,
     'imports': 123.617,
     'primary_production': 2761.8,
     'dependency': -0.075996,
    })

In [27]:
cz_2009_bioenergy
cz_2009_bioenergy.name = ('CZ', 2009, 'bioenergy')

In [28]:
pd.testing.assert_series_equal(cz_2009_bioenergy, result_cz_2009_bioenergy)